<a href="https://colab.research.google.com/github/Hussein-Ahmad-94/Tasks/blob/main/Trial_MTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam
df=pd.read_csv("/content/train.csv",parse_dates=["Date"],index_col=[0])
print(df.index.freq)
df.shape
df.head()
df.tail()

x1 = df.iloc[:,0]
x2 = df.iloc[:,1]
x3 = df.iloc[:,2]
x4 = df.iloc[:,3]
x5 = df.iloc[:,4]

input_data = np.concatenate([x2,x3,x4,x5], axis=-1)


input_data

None


array([362.980774, 360.096161, 355.769226, ..., 105.257835, 105.637512,
       106.187027])

In [ ]:
import numpy as np

# Assuming x2, x3, x4, x5 are pandas Series
# Convert them to numpy arrays using the .values attribute

x2_array = x2.values.reshape(-1, 1)  # Reshape to a column vector
x3_array = x3.values.reshape(-1, 1)
x4_array = x4.values.reshape(-1, 1)
x5_array = x5.values.reshape(-1, 1)

future_steps = 3

# Now concatenate the arrays
y_true = np.concatenate([x2_array[-future_steps:, :], x3_array[-future_steps:, :], x4_array[-future_steps:, :], x5_array[-future_steps:, :]], axis=-1)


In [ ]:
# Define a custom loss function
def custom_loss(y_true, y_pred):
    y_pred_expanded = K.expand_dims(y_pred, -1)  # Expanding the last dimension of y_pred
    loss_x2 = K.mean(K.square(y_pred_expanded - y_true[:, :, :1]))  # Referencing the second feature from y_true
    loss_x3 = K.mean(K.square(y_pred_expanded - y_true[:, :, 1:2]))  # Referencing the third feature from y_true
    loss_x4 = K.mean(K.square(y_pred_expanded - y_true[:, :, 2:3]))  # Referencing the fourth feature from y_true
    loss_x5 = K.mean(K.square(y_pred_expanded - y_true[:, :, 3:4]))  # Referencing the fifth feature from y_true
    total_loss = loss_x2 + loss_x3 + loss_x4 + loss_x5
    return total_loss


In [ ]:
# Build the model
input_layer = Input(shape=(time_steps, 2))
x = LSTM(64, return_sequences=True)(input_layer)
x = LSTM(64)(x)
output_layer = Dense(future_steps)(x)


In [ ]:
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with the custom loss function
model.compile(optimizer=Adam(), loss=custom_loss)



In [ ]:
# Assuming input_data is 1-dimensional
# Reshape it to have two dimensions: (number of samples, number of features)
input_data = input_data.reshape((input_data.shape[0], 1))

# Now, add a third dimension for timesteps (assuming you have one timestep per sample)
input_data = np.expand_dims(input_data, axis=1)

# Train the model
model.fit(input_data, y_true, epochs=3, batch_size=32)


Epoch 1/3


ValueError: ignored

**Cutt**

In [ ]:
test_split=round(len(df)*0.20)
test_split
df.iloc[:,1]
df_for_training=df[:-test_split]
df_for_testing=df[-test_split:]
print(df_for_training.shape)
print(df_for_testing.shape)
scaler = MinMaxScaler(feature_range=(0,1))
df_for_training_scaled = scaler.fit_transform(df_for_training)
df_for_testing_scaled=scaler.transform(df_for_testing)
df_for_training_scaled[:35]
df_for_training_scaled.shape
df_for_testing_scaled.shape

In [ ]:
def createXY(dataset,n_past):
    dataX = []
    dataY = []
    for i in range(n_past, len(dataset)):
            dataX.append(dataset[i - n_past:i, 0:dataset.shape[1]])
            dataY.append(dataset[i,0])
    return np.array(dataX),np.array(dataY)
trainX,trainY=createXY(df_for_training_scaled,30)
testX,testY=createXY(df_for_testing_scaled,30)
print(trainX.shape)
print(trainY.shape)
print(testX.shape)
print(testY.shape)
trainX[1]
trainX[0]
trainY[0]
trainX[1]
trainY[1]
print("trainX Shape-- ",trainX.shape)
print("trainY Shape-- ",trainY.shape)
print("testX Shape-- ",testX.shape)
print("testY Shape-- ",testY.shape)
print("trainX[0]-- \n",trainX[0])
print("\ntrainY[0]-- ",trainY[0])

In [ ]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
# Generate some random data for demonstration
data_size = 1000
time_steps = 10
future_steps = 3

x1 = np.random.rand(data_size, time_steps, 1)
x2 = np.random.rand(data_size, time_steps, 1)
x3 = np.random.rand(data_size, future_steps, 1)

# Concatenate x1 and x2 along the last dimension to form the input_data
input_data = np.concatenate([x1, x2], axis=-1)

# Generate true labels (y_true) by concatenating the last future_steps of x1 and x2
y_true = np.concatenate([x1[:, -future_steps:, :], x2[:, -future_steps:, :]], axis=-1)

# Define a custom loss function
def custom_loss(y_true, y_pred):
    y_pred_expanded = K.expand_dims(y_pred, -1)  # Expanding the last dimension of y_pred
    loss_x1 = K.mean(K.square(y_pred_expanded - y_true[:, :, :1]))  # Referencing the first feature from y_true
    loss_x2 = K.mean(K.square(y_pred_expanded - y_true[:, :, 1:]))  # Referencing the second feature from y_true
    total_loss = loss_x1 + loss_x2
    return total_loss

# Build the model
input_layer = Input(shape=(time_steps, 2))
x = LSTM(64, return_sequences=True)(input_layer)
x = LSTM(64)(x)
output_layer = Dense(future_steps)(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with the custom loss function
model.compile(optimizer=Adam(), loss=custom_loss)

# Train the model
model.fit(input_data, y_true, epochs=3, batch_size=32)


Epoch 1/3
32/32 [==============================] - 6s 15ms/step - loss: 0.2257
Epoch 2/3
32/32 [==============================] - 0s 15ms/step - loss: 0.1683
Epoch 3/3
32/32 [==============================] - 1s 19ms/step - loss: 0.1664


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

# Generate some random data for demonstration
data_size = 1000
time_steps = 10
future_steps = 3

x1 = np.random.rand(data_size, time_steps, 1)
x2 = np.random.rand(data_size, time_steps, 1)
x3 = np.random.rand(data_size, future_steps, 1)

# Concatenate x1 and x2 along the last dimension to form the input_data
input_data = np.concatenate([x1, x2], axis=-1)

# Generate true labels (y_true) by concatenating the last future_steps of x1 and x2
y_true = np.concatenate([x1[:, -future_steps:, :], x2[:, -future_steps:, :]], axis=-1)

# Split the data into training and testing sets without shuffling
input_data_train, input_data_test, y_true_train, y_true_test = train_test_split(input_data, y_true, test_size=0.2, shuffle=False)

# Define a custom loss function
def custom_loss(y_true, y_pred):
    y_pred_expanded = K.expand_dims(y_pred, -1)
    loss_x1 = K.mean(K.square(y_pred_expanded - y_true[:, :, :1]))
    loss_x2 = K.mean(K.square(y_pred_expanded - y_true[:, :, 1:]))
    total_loss = loss_x1 + loss_x2
    return total_loss

# Build the model
input_layer = Input(shape=(time_steps, 2))
x = LSTM(64, return_sequences=True)(input_layer)
x = LSTM(64)(x)
output_layer = Dense(future_steps)(x)

model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with the custom loss function
model.compile(optimizer=Adam(), loss=custom_loss)

# Train the model on the training data
model.fit(input_data_train, y_true_train, epochs=3, batch_size=32)

# Evaluate the model on the testing data
loss = model.evaluate(input_data_test, y_true_test)
print(f"Test Loss: {loss}")


Epoch 1/3
25/25 [==============================] - 2s 6ms/step - loss: 0.2583
Epoch 2/3
25/25 [==============================] - 0s 7ms/step - loss: 0.1624
Epoch 3/3
7/7 [==============================] - 1s 3ms/step - loss: 0.1661
Test Loss: 0.16611069440841675


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

# Load the dataset
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')

# Select relevant columns
df_input = df[['Appliances', 'T_out', 'RH_1', 'Visibility']]

"""# Split the data into features (X) and target (y)
X = df_input[['T_out', 'RH_1', 'Visibility']].values
y = df_input['Appliances'].values"""

"""# Normalize the features using Min-Max scaling
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)"""
X_scaled = X
# Define time_steps and future_steps
time_steps = 10  # Number of time steps in the input sequence
future_steps = 3  # Number of future steps to predict

"""# Create sequences for input data and labels
input_data, y_true = [], []
for i in range(len(X_scaled) - time_steps - future_steps + 1):
    input_data.append(X_scaled[i:i + time_steps])
    y_true.append(X_scaled[i + time_steps:i + time_steps + future_steps])

input_data = np.array(input_data)
y_true = np.array(y_true)
"""


'# Create sequences for input data and labels\ninput_data, y_true = [], []\nfor i in range(len(X_scaled) - time_steps - future_steps + 1):\n    input_data.append(X_scaled[i:i + time_steps])\n    y_true.append(X_scaled[i + time_steps:i + time_steps + future_steps])\n\ninput_data = np.array(input_data)\ny_true = np.array(y_true)\n'

In [ ]:
data_X1 = np.array(df_input["Appliances"])
data_X2 = np.array(df_input["T_out"])
data_X3 = np.array(df_input["RH_1"])
data_X4 = np.array(df_input["Visibility"])

# Step 1 : convert to [rows, columns] structure
data_X1= data_X1.reshape( (len(data_X1)), 1)
data_X2= data_X2.reshape( (len(data_X2)), 1)
data_X3= data_X3.reshape( (len(data_X3)), 1)
data_X4= data_X3.reshape( (len(data_X4)), 1)

In [ ]:
print ("data_X1.shape", data_X1.shape)
print ("data_X2.shape", data_X2.shape)
print ("data_X3.shape", data_X3.shape)
print ("data_X4.shape", data_X4.shape)

data_X1.shape (19735, 1)
data_X2.shape (19735, 1)
data_X3.shape (19735, 1)
data_X4.shape (19735, 1)


In [ ]:
import tensorflow as tf
from keras import Model
from tensorflow.keras.utils import plot_model
from keras.layers import Input, Dense, Bidirectional, LSTM, RepeatVector, TimeDistributed
from sklearn.preprocessing import MinMaxScaler
from numpy import array, hstack
import numpy as np
# Step 3 : horizontally stack columns
dataset_stacked = hstack((data_X1,data_X2,data_X3,data_X4))
dataset_stacked = np.array(dataset_stacked)


In [ ]:
print("dataset_stacked.shape", dataset_stacked.shape)
print("final dataset => ", dataset_stacked)

dataset_stacked.shape (19735, 4)
final dataset =>  [[ 60.           6.6         47.59666667  47.59666667]
 [ 60.           6.48333333  46.69333333  46.69333333]
 [ 50.           6.36666667  46.3         46.3       ]
 ...
 [270.          22.46666667  46.59666667  46.59666667]
 [420.          22.33333333  46.99        46.99      ]
 [430.          22.2         46.6         46.6       ]]


In [ ]:
# Prepare the dataset
input_timesteps=10
input_features = 3
output_timesteps = 1
output_features = 1

data_Y = []
data_X = []

for i in range(0, ((len(data_X1))-(input_timesteps+output_timesteps))):
    print("===================")
    tmpY2d = []
    for i_row in range(i, (i+ output_timesteps)):
        tmpYr = []
        print(i_row)
        for i_col in range(0, output_features):
            tmpYr.append(dataset_stacked[i_row][i_col])
        tmpY2d.append(tmpYr)
    data_Y.append(tmpY2d)
    print("-------------------------")
    tmpX2d= []
    for j_row in range( (i+ output_timesteps),(i+ output_timesteps + input_timesteps) ):
        tmpXr = []
        print(j_row)
        for j_col in range(0, input_features):
            tmpXr.append(dataset_stacked[j_row][j_col])

        tmpX2d.append(tmpXr)
    data_X.append(tmpX2d)



5091
5092
5093
5094
5095
5086
-------------------------
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5087
-------------------------
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5088
-------------------------
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5089
-------------------------
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5090
-------------------------
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5091
-------------------------
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5092
-------------------------
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5093
-------------------------
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5094
-------------------------
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5095
-------------------------
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5096
-------------------------
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5097
-------------------------
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
509

KeyboardInterrupt: ignored

In [ ]:
data_X = np.array(data_X)
data_Y = np.array(data_Y)
print("shape of input data", data_X.shape)
print("input data => ", data_X)
print("shape of output data", data_Y.shape)
print("output data => ", data_Y)

shape of input data (19724, 10, 3)
input data =>  [[[ 60.           6.48333333  46.69333333]
  [ 50.           6.36666667  46.3       ]
  [ 50.           6.25        46.06666667]
  ...
  [ 60.           5.93333333  45.5975    ]
  [ 70.           5.95        46.09      ]
  [230.           5.96666667  45.86333333]]

 [[ 50.           6.36666667  46.3       ]
  [ 50.           6.25        46.06666667]
  [ 60.           6.13333333  46.33333333]
  ...
  [ 70.           5.95        46.09      ]
  [230.           5.96666667  45.86333333]
  [580.           5.98333333  46.39666667]]

 [[ 50.           6.25        46.06666667]
  [ 60.           6.13333333  46.33333333]
  [ 50.           6.01666667  46.02666667]
  ...
  [230.           5.96666667  45.86333333]
  [580.           5.98333333  46.39666667]
  [430.           6.          48.        ]]

 ...

 [[100.          22.8         45.73      ]
  [ 90.          22.83333333  45.79      ]
  [100.          22.86666667  45.93333333]
  ...
  [ 90.    

In [ ]:
data_X[0]

array([[ 60.        ,   6.48333333,  46.69333333],
       [ 50.        ,   6.36666667,  46.3       ],
       [ 50.        ,   6.25      ,  46.06666667],
       [ 60.        ,   6.13333333,  46.33333333],
       [ 50.        ,   6.01666667,  46.02666667],
       [ 60.        ,   5.9       ,  45.76666667],
       [ 60.        ,   5.91666667,  45.56      ],
       [ 60.        ,   5.93333333,  45.5975    ],
       [ 70.        ,   5.95      ,  46.09      ],
       [230.        ,   5.96666667,  45.86333333]])

In [ ]:
data_Y[2]

array([[50.]])

In [ ]:
# Split the data into training and testing sets without shuffling
input_data_train, input_data_test, y_true_train, y_true_test = train_test_split(input_data, y_true, test_size=0.2, shuffle=False)



In [ ]:
print("shape X train", input_data_train.shape)
print("shape y train", y_true_train.shape)
print("shape X test", input_data_test.shape)
print("shape y test", y_true_test.shape)


shape X train (15778, 10, 3)
shape y train (15778, 3, 3)
shape X test (3945, 10, 3)
shape y test (3945, 3, 3)


In [ ]:
print("shape X train", input_data_train[0])

shape X train [[ 6.6        47.59666667 63.        ]
 [ 6.48333333 46.69333333 59.16666667]
 [ 6.36666667 46.3        55.33333333]
 [ 6.25       46.06666667 51.5       ]
 [ 6.13333333 46.33333333 47.66666667]
 [ 6.01666667 46.02666667 43.83333333]
 [ 5.9        45.76666667 40.        ]
 [ 5.91666667 45.56       40.        ]
 [ 5.93333333 45.5975     40.        ]
 [ 5.95       46.09       40.        ]]


In [ ]:
print("shape y train", y_true_train[0])

shape y train [[ 5.96666667 45.86333333 40.        ]
 [ 5.98333333 46.39666667 40.        ]
 [ 6.         48.         40.        ]]


In [ ]:
# Define the custom loss
def custom_loss(y_true, y_pred):
    loss_x1 = K.mean(K.square(y_pred[:, :, 0] - y_true[:, :, 0]))
    loss_x2 = K.mean(K.square(y_pred[:, :, 1] - y_true[:, :, 1]))
    loss_x3 = K.mean(K.square(y_pred[:, :, 2] - y_true[:, :, 2]))
    total_loss = loss_x1 + loss_x2 + loss_x3
    return total_loss

In [ ]:
# Build the model
input_layer = Input(shape=(time_steps, X_scaled.shape[1]))
x = LSTM(64, return_sequences=True)(input_layer)
x = LSTM(64)(x)

# Build the model with 3 units in the output layer to match the number of features
output_layer = Dense(3)(x)
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with the custom loss function
model.compile(optimizer=Adam(), loss=custom_loss)

# Cast y_true to float32 to match the data type of y_pred
y_true_train = y_true_train.astype('float32')

# Train the model on the training data
model.fit(input_data_train, y_true_train, epochs=3, batch_size=32)

**Validation of the notion**

In [ ]:
import pandas as pd

data = {'_tempm': [72, 73, 74, 75, 76, 77, 78, 79, 80, 81],
        'humidity': [50, 52, 55, 56, 58, 60, 62, 63, 65, 68]}
df = pd.DataFrame(data)

In [ ]:
sequence_length = 4  # Number of time steps in each sequence


In [ ]:
# Create sequences and corresponding labels
sequences = []
labels = []
for i in range(len(df) - sequence_length):
    seq = df[i:i+sequence_length]
    label = df[i+sequence_length][1]  # '_tempm' column index
    sequences.append(seq)
    labels.append(label)

# Convert to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)

KeyError: ignored

In [ ]:
import pandas as pd
import numpy as np

data = {'_tempm': [72, 73, 74, 75, 76, 77, 78, 79, 80, 81],
        'humidity': [50, 52, 55, 56, 58, 60, 62, 63, 65, 68]}
df = pd.DataFrame(data)
sequence_length = 4  # Number of time steps in each sequence

# Create sequences and corresponding labels
sequences = []
labels = []

for i in range(len(df) - sequence_length):
    seq = df[i:i+sequence_length]
    label = df[i+sequence_length][1]  # 'humidity' column index
    sequences.append(seq)
    labels.append(label)

# Convert to numpy arrays
sequences = np.array(sequences)
labels = np.array(labels)


KeyError: ignored

In [ ]:
import numpy as np

# Create a synthetic dataset with 7 columns (you can replace this with your real dataset)
num_rows = 100
num_columns = 7
synthetic_data = np.random.rand(num_rows, num_columns)

# Define the sequence length
sequence_length = 5

# Create sequences and corresponding labels
sequences = []
labels = []

for i in range(len(synthetic_data) - sequence_length):
    # Create a sequence of length 'sequence_length' from 'synthetic_data'
    seq = synthetic_data[i:i+sequence_length]

    # Extract the label from the 7th column (assuming 0-based indexing)
    label = synthetic_data[i+sequence_length][6]

    # Append the sequence and label to their respective lists
    sequences.append(seq)
    labels.append(label)

# Convert the lists to NumPy arrays for further processing
sequences = np.array(sequences)
labels = np.array(labels)

# Print the generated sequences and labels for illustration
print("Sequences:")
print(sequences)
print("Labels:")
print(labels)


Sequences:
[[[0.3959976  0.68271796 0.67950056 ... 0.9176165  0.28581863 0.55062091]
  [0.45234044 0.77326686 0.52525395 ... 0.5200104  0.94191259 0.97990763]
  [0.40302592 0.63691622 0.28331981 ... 0.79557395 0.58954641 0.09583094]
  [0.04241328 0.0679106  0.76058118 ... 0.7389199  0.15433037 0.83866857]
  [0.06395503 0.14158878 0.3940531  ... 0.46276749 0.82175677 0.38595353]]

 [[0.45234044 0.77326686 0.52525395 ... 0.5200104  0.94191259 0.97990763]
  [0.40302592 0.63691622 0.28331981 ... 0.79557395 0.58954641 0.09583094]
  [0.04241328 0.0679106  0.76058118 ... 0.7389199  0.15433037 0.83866857]
  [0.06395503 0.14158878 0.3940531  ... 0.46276749 0.82175677 0.38595353]
  [0.5128872  0.25797919 0.99670109 ... 0.3743473  0.78548253 0.39713662]]

 [[0.40302592 0.63691622 0.28331981 ... 0.79557395 0.58954641 0.09583094]
  [0.04241328 0.0679106  0.76058118 ... 0.7389199  0.15433037 0.83866857]
  [0.06395503 0.14158878 0.3940531  ... 0.46276749 0.82175677 0.38595353]
  [0.5128872  0.2579791